# First Steps

This is an example of what you can already with GMT/Python. 
We'll mostly reproduce some examples from the [GMT Tutorial](http://gmt.soest.hawaii.edu/doc/latest/GMT_Tutorial.html#session-one).

## Loading the library

The GMT modules are available as functions and classes in the `gmt` Python package. 
See the [API Reference](api.html) for a list of all available functions and classes.

First, import the `gmt` package:

In [ ]:
import gmt

## A first map

All figure generation in GMT/Python is handled by the [gmt.Figure](api/gmt.Figure.html) class. 
It has methods to add layers to your figure (the GMT `ps*` modules).

We start a new figure by creating an instance of `gmt.Figure`:

In [ ]:
fig = gmt.Figure()

Add elements to the figure using its methods. For example, lets add the coastlines of Central America using a 6 inch wide Mercator map.

In [ ]:
fig.pscoast(R='-90/-70/0/20', J='M6i', G='chocolate', 
            B='afg')

Most methods of `gmt.Figure` accept the same arguments as the GMT command-line programs. A general rule is that you can pass any argument as a string value. If an argument has no options (like `-P`), use a `True` value for it instead.

<div class="alert alert-success">
Experienced GMT users will notice that there is no output redirecting to a PostScript file. 
This is all thanks to the new GMT [modern execution mode](http://gmt.soest.hawaii.edu/projects/gmt/wiki/Modernization) that will be part of the future 6.0 release of GMT.
</div>

You can see a preview of the figure directly in the [Jupyter notebook](http://jupyter.org) using `fig.show()`.

In [ ]:
fig.show()

To open a PDF preview of the figure using your default PDF viewer use:

```python
fig.show(external=True)
```

This is useful when using the Python shell or IPython terminal app. 
However, **this command will not interrupt your Python process**. 
So using it in a Python script will not work because the script will finish and delete the generated previews.
Use `fig.savefig` to save the figure to a file instead (see below).

## Aliases

GMT/Python defines long-form aliases for the command-line options to make the code more Pythonic and readble. The docstrings of the functions and methods list the possible aliases:

In [ ]:
help(fig.pscoast)

Some options also accept values other than strings. For example, `R` (or `region`) can be passed as a list. This makes it easier to generate such arguments programatically.

We can recreate the figure above using the aliases instead:

In [ ]:
# Create a new figure
fig_alias = gmt.Figure()
fig_alias.pscoast(
    region=[-90, -70, 0, 20], projection='M6i', shorelines=True,
    land='chocolate', water='skyblue', frame='afg')
fig_alias.show()

## Saving the figure

Unlike the GMT command-line interface, **no figure file was generated until you ask for one**.

You can call `fig.psconvert` to save the figure to a file:

In [ ]:
# Format 'f' means a PDF
fig.psconvert(prefix='first-steps-central-america', fmt='f')

or use the matplotlib-like method `fig.savefig`:

In [ ]:
fig.savefig('first-steps-central-america.png')

## Plot point data

We can use `gmt.Figure.psxy` to plot data on our map.

Lets create some data to plot using numpy:

In [ ]:
import numpy as np

In [ ]:
# See the random number generator so we always 
# get the same numbers
np.random.seed(42)

lon = np.random.uniform(150, 240, 30)
lat = np.random.uniform(-30, 60, 30)

Save the data to a file in 2 columns:

In [ ]:
np.savetxt('first-steps-data.txt', np.transpose([lon, lat]))

Now we can plot the data using `Figure.psxy` and passing the file name as the `data` argument:

In [ ]:
fig_globe = gmt.Figure()
fig_globe.pscoast(
    region='g', projection='G200/30/6i', frame='ag', 
    resolution='i', area_thresh=5000, land='white', 
    water='DarkTurquoise')
fig_globe.psxy(data='first-steps-data.txt', style='s0.3c',
               color='black')
fig_globe.show()

GMT ships some sample data that can be accessed by passing `@data_file` as the `data` argument. 

For example, we can plot the earthquake epicenters from the `tut_quakes.ngdc` sample dataset:

In [ ]:
fig_quakes = gmt.Figure()
fig_quakes.pscoast(
    region=[130, 150, 35, 50], projection='M6i', 
    frame='afg', shorelines=True, land='gray', 
    water='lightblue')
fig_quakes.psxy(data='@tut_quakes.ngdc', style='c0.3c', 
                color='blue', pen='faint', columns=[4, 3])
fig_quakes.show()

## Coming soon

Here are a few of the features we are working on:

* Data input using numpy arrays, not just files.
* More GMT modules (`grdcontour`, etc).